In [1]:
import arcpy
import math
import pandas as pd
import numpy as np

In [2]:
# set the working envrionment
arcpy.env.workspace = '../'
arcpy.env.overwriteOutput = True

In [3]:
# import the shp of North_China_Plain
in_fe = '../North_china_plain.shp'

# get the spatial-reference
sr = arcpy.da.Describe(in_fe)['spatialReference']

# set the output coordinate system so we do not need 
# to define each Geometry-object individualy
arcpy.env.outputCoordinateSystem = sr

In [4]:
# convert the shp into an arcpy.Geometry object
Ncp_geo = arcpy.CopyFeatures_management(in_fe,arcpy.Geometry())[0]

In [5]:
# get the area constrain, here we want to divide the shp into 10 equal 
# area parts
Area_constrain = Ncp_geo.area/10

# get the central point of the shp
Center_pt = Ncp_geo.centroid

In [6]:
# here we calculate the longest distance between in_fe and its center
dist = max([arcpy.PointGeometry(pt).distanceTo(Center_pt) for pt in Ncp_geo[0]])

In [7]:
# draw a circle that use r=dist and center=centroid
buffer = arcpy.PointGeometry(Center_pt).buffer(dist)
arcpy.CopyFeatures_management(buffer,'../Center_buffer.shp')

<Result '..\\Center_buffer.shp'>

### Buffer to line  --> line to point; 

In [8]:
# buffer to line
arcpy.PolygonToLine_management(in_features="../Center_buffer.shp", 
                               out_feature_class="../Center_buffer_line.shp", 
                               neighbor_option="IDENTIFY_NEIGHBORS")


<Result '..\\Center_buffer_line.shp'>

In [9]:
# Buffer_line to point
pnt_coors = []

# Use searchCursor to dig into each pnt of the buffer_line
for row in  arcpy.da.SearchCursor('../Center_buffer_line.shp',['SHAPE@']):
    for part in row[0]:     
        for pnt in part:
            #Print x,y coordinates of current point
            pnt_coors.append((pnt.X, pnt.Y))

# 
Buffer_pt = [arcpy.PointGeometry(arcpy.Point(*coords)) for coords in pnt_coors]
arcpy.CopyFeatures_management(Buffer_pt,'../Center_buffer_line_pt.shp')

<Result '..\\Center_buffer_line_pt.shp'>

### Iterativelly create a sector polygon 

In [11]:
Pt_percent = {}

for i in range(0,len(pnt_coors),10):
    
    # construct the sector using points from the buffer_circle and the center_pt
    sector_pt = [arcpy.Point(*coords) for coords in pnt_coors[:i+1]] + [Center_pt]
    sector_polyton = arcpy.Polygon(arcpy.Array(sector_pt))
    intersect = sector_polyton.intersect(Ncp_geo,4)
    
    # calculate the percentage of each sector    
    Area_pct = intersect.area/Ncp_geo.area * 100
    # record its FID with percentage
    Pt_percent[i] = Area_pct
    
    print(f'{i+1:04}/{len(pnt_coors)} ======> Area_Percent: {Area_pct:.2f}')
    

0001/6281 ======> Area_Percent: 0.00
0011/6281 ======> Area_Percent: 0.33
0021/6281 ======> Area_Percent: 0.67
0031/6281 ======> Area_Percent: 1.01
0041/6281 ======> Area_Percent: 1.39
0051/6281 ======> Area_Percent: 1.78
0061/6281 ======> Area_Percent: 2.17
0071/6281 ======> Area_Percent: 2.56
0081/6281 ======> Area_Percent: 2.94
0091/6281 ======> Area_Percent: 3.34
0101/6281 ======> Area_Percent: 3.74
0111/6281 ======> Area_Percent: 4.15
0121/6281 ======> Area_Percent: 4.55
0131/6281 ======> Area_Percent: 4.96
0141/6281 ======> Area_Percent: 5.35
0151/6281 ======> Area_Percent: 5.74
0161/6281 ======> Area_Percent: 6.11
0171/6281 ======> Area_Percent: 6.46
0181/6281 ======> Area_Percent: 6.81
0191/6281 ======> Area_Percent: 7.15
0201/6281 ======> Area_Percent: 7.47
0211/6281 ======> Area_Percent: 7.77
0221/6281 ======> Area_Percent: 8.06
0231/6281 ======> Area_Percent: 8.33
0241/6281 ======> Area_Percent: 8.60
0251/6281 ======> Area_Percent: 8.86
0261/6281 ======> Area_Percent: 9.11
0

2171/6281 ======> Area_Percent: 27.81
2181/6281 ======> Area_Percent: 27.96
2191/6281 ======> Area_Percent: 28.11
2201/6281 ======> Area_Percent: 28.26
2211/6281 ======> Area_Percent: 28.42
2221/6281 ======> Area_Percent: 28.60
2231/6281 ======> Area_Percent: 28.80
2241/6281 ======> Area_Percent: 29.03
2251/6281 ======> Area_Percent: 29.28
2261/6281 ======> Area_Percent: 29.54
2271/6281 ======> Area_Percent: 29.81
2281/6281 ======> Area_Percent: 30.07
2291/6281 ======> Area_Percent: 30.32
2301/6281 ======> Area_Percent: 30.55
2311/6281 ======> Area_Percent: 30.77
2321/6281 ======> Area_Percent: 30.99
2331/6281 ======> Area_Percent: 31.21
2341/6281 ======> Area_Percent: 31.44
2351/6281 ======> Area_Percent: 31.69
2361/6281 ======> Area_Percent: 31.93
2371/6281 ======> Area_Percent: 32.17
2381/6281 ======> Area_Percent: 32.41
2391/6281 ======> Area_Percent: 32.65
2401/6281 ======> Area_Percent: 32.90
2411/6281 ======> Area_Percent: 33.15
2421/6281 ======> Area_Percent: 33.40
2431/6281 ==

4331/6281 ======> Area_Percent: 71.71
4341/6281 ======> Area_Percent: 71.92
4351/6281 ======> Area_Percent: 72.12
4361/6281 ======> Area_Percent: 72.32
4371/6281 ======> Area_Percent: 72.52
4381/6281 ======> Area_Percent: 72.73
4391/6281 ======> Area_Percent: 72.94
4401/6281 ======> Area_Percent: 73.15
4411/6281 ======> Area_Percent: 73.36
4421/6281 ======> Area_Percent: 73.57
4431/6281 ======> Area_Percent: 73.78
4441/6281 ======> Area_Percent: 74.00
4451/6281 ======> Area_Percent: 74.21
4461/6281 ======> Area_Percent: 74.43
4471/6281 ======> Area_Percent: 74.64
4481/6281 ======> Area_Percent: 74.85
4491/6281 ======> Area_Percent: 75.06
4501/6281 ======> Area_Percent: 75.27
4511/6281 ======> Area_Percent: 75.49
4521/6281 ======> Area_Percent: 75.71
4531/6281 ======> Area_Percent: 75.93
4541/6281 ======> Area_Percent: 76.14
4551/6281 ======> Area_Percent: 76.36
4561/6281 ======> Area_Percent: 76.58
4571/6281 ======> Area_Percent: 76.80
4581/6281 ======> Area_Percent: 76.99
4591/6281 ==

In [38]:
equal_area_index = []

for i in range(10,91,10):
    
    diff = [abs(i - j) for j in Pt_percent.values()]
    
    min_idx    = diff.index(min(diff))
    circle_idx = list(Pt_percent.keys())[min_idx]
    
    equal_area_index.append(circle_idx)
    print(circle_idx)

# add the first and last index into the index_list
equal_area_index = [0] + equal_area_index + [list(Pt_percent.keys())[-1]]

300
1160
2280
2710
3080
3640
4250
5010
5980


In [43]:
# get each individual sector
for i in range(len(equal_area_index) - 1):
    
    start_idx = equal_area_index[i]
    end_idx   = equal_area_index[i + 1] + 1
    
    # construct the sector using points from the buffer_circle and the center_pt
    sector_pt = [arcpy.Point(*coords) for coords in pnt_coors[start_idx:end_idx]] + [Center_pt]
    sector_polyton = arcpy.Polygon(arcpy.Array(sector_pt))
    # compute the intersect
    intersect = sector_polyton.intersect(Ncp_geo,4)
    
    arcpy.CopyFeatures_management(intersect,f'../Sector_{i}')
    
    print(f'Sector intersection {i} finished!')

Sector intersection 0 finished!
Sector intersection 1 finished!
Sector intersection 2 finished!
Sector intersection 3 finished!
Sector intersection 4 finished!
Sector intersection 5 finished!
Sector intersection 6 finished!
Sector intersection 7 finished!
Sector intersection 8 finished!
Sector intersection 9 finished!


In [46]:
# combine all sectors into one multipolygon
sector_shps = arcpy.ListFeatureClasses('Sec*')

arcpy.Merge_management(sector_shps,'../Sector_combine.shp')


<Result '..\\Sector_combine.shp'>

['Sector_0.shp',
 'Sector_1.shp',
 'Sector_2.shp',
 'Sector_3.shp',
 'Sector_4.shp',
 'Sector_5.shp',
 'Sector_6.shp',
 'Sector_7.shp',
 'Sector_8.shp',
 'Sector_9.shp']